In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir+'../../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# dx = 250m; Np = 50M
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
###########################
#Plotting
plotting=False #KEEP FALSE WHEN JOB ARRAYS IS RUNNING
plotting=True

In [ ]:
#READING BACK IN
# import pickle
# dir2 = dir + f'Project_Algorithms/Domain_Profiles/'
# input_file = dir2 + f'mean_lfc_{res}_{t_res}_{Np_str}.pkl'

# with open(input_file, 'rb') as f:
#     mean_LFC = pickle.load(f)
# print(mean_LFC)

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
mean_LFC=MeanLFC
print(f"Mean LFC is: {MeanLFC}\n")


def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

In [ ]:
#thresholds
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6
dir2=dir+'Project_Algorithms/Domain_Profiles/'

In [ ]:
# def GetPropertyProfile(type):
#     if type == "general":
#         property_input_file = dir2 + f'OUTPUT/general_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'
#     elif type == "cloudy":
#         property_input_file = dir2 + f'OUTPUT/cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'

#     output = {}
#     with h5py.File(property_input_file, 'r') as f:
#         for var in ['w', 'qv', 'th']:
#             profile = f[f'profile_{var}'][:]
#             try:
#                 profile_squares = f[f'profile_{var}_squares'][:]
#                 profile_SE = ProfileStandardError(profile, profile_squares)
#                 # profile_SE = ProfileStandardDeviation(profile, profile_squares)
#             except:
#                 #IF SE NOT AVAILABLE
#                 dummy=profile.copy()
#                 dummy[:, 0] = np.zeros(len(dummy[:, 0]))
#                 dummy=dummy[dummy[:,1]>0]
#                 profile_SE=dummy
            
#             output[f'profile_{var}'] = profile
#             output[f'profile_{var}_SE'] = profile_SE


#     return output

def GetBudgetProfile(var_type,type):
    if type=='general':
        input_file = dir2+f'OUTPUT/{var_type}_BUDGET_general_eulerian_profiles_{res}_{t_res}_{Np_str}.h5' 
    elif type=='cloudy':
        input_file = dir2+f'OUTPUT/{var_type}_BUDGET_cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}.h5' 
    return input_file

def LoadProfiles(input_file, budget_type):
    with h5py.File(input_file, 'r') as f:
        keys_list=[]
        for key in f.keys():
            if budget_type in key:
                globals()[key] = np.array(f[key])
                if '_squares' not in key:
                    keys_list.append(key)
        for key in keys_list:
            try:
                globals()[f"{key}_SE"]=ProfileStandardError(globals()[f"{key}"],globals()[f"{key}_squares"])
                # globals()[f"{key}_SE"]=ProfileStandardDeviation(globals()[f"{key}"],globals()[f"{key}_squares"])

            except:
                #IF SE NOT AVAILABLE
                dummy=globals()[f"{key}"].copy()
                dummy[:, 0] = np.zeros(len(dummy[:, 0]))
                dummy=dummy[dummy[:,1]>0]
                globals()[f"{key}_SE"]=dummy

In [ ]:
def PlotBudgetProfiles(axis,vars,budget_unit):
    import h5py

    def averaged_profiles(var):
        profile = globals()[f"profile_{var}"]
        out_var = profile[(profile[:, 1] > 1)]
        out_var = np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T
        return out_var

    dim = '1km'
    label = 1

    colors = [
        'blue', 'orange', 'green', 'red',
        'purple', 'brown', 'pink', 'grey'
    ]

    def plotting_func(out_var, axis, label, color, linestyle):
        axis.plot(out_var[:, 0], out_var[:, 1], label=label, color=color, linestyle=linestyle)
        axis.grid(True)

    for type in ["general", "cloudy"]:
        print(f'currently on type {type}')

        if any('wb' in var for var in vars):
            budget_type = 'wb'
            BUDGET_TYPE = 'W'
        elif any('qvb' in var for var in vars):
            budget_type = 'qvb'
            BUDGET_TYPE = 'QV'
        elif any('ptb' in var for var in vars):
            budget_type = 'ptb'
            BUDGET_TYPE = 'TH'
        input_file = GetBudgetProfile(BUDGET_TYPE, type)
        LoadProfiles(input_file, budget_type)

        linestyle = 'dashed' if type == 'general' else 'solid'

        for var in vars:
            globals()[f"out_{var}"] = averaged_profiles(var)

        for var, xlabel, color in zip(vars, vars, colors):
            profile = globals()[f"out_{var}"].copy()
            legend_label = xlabel if type == "cloudy" else None
            plotting_func(profile, axis, label=legend_label, color=color, linestyle=linestyle)

            profile_SE = globals()[f"profile_{var}_SE"]
            axis.fill_betweenx(profile[:, 1],
                               profile[:, 0] - profile_SE[:, 0],
                               profile[:, 0] + profile_SE[:, 0],
                               color=color, alpha=0.1)

    axis.set_ylim(bottom=0, top=20)
    axis.set_ylabel('z (km)')
    axis.set_xlabel(budget_unit)
    axis.legend(fontsize='small')

    #SCIENTIFIC NOTATION X AXIS
    apply_scientific_notation([axis])

    #ADDING CLOUDBASE AND LFC
    axline_lw = 1.2
    axis.axhline(cloudbase, color='purple', linestyle='dashed', lw=axline_lw, zorder=-100)
    axis.axhline(mean_LFC / 1000, color='green', linestyle='dashed', lw=axline_lw, zorder=-100)

In [ ]:
def SavePlot(fig, res, t_res, Np_str, dir2, limit_y=False):
    # Define output directory
    subdir_name = f'{res}_{t_res}_{Np_str}'
    output_dir = os.path.join(dir2,'PLOTS', 'Updraft_Budgets', subdir_name)
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Choose file name
    if not limit_y:
        filename = f'Budget_Updraft_Profiles_{res}_{t_res}_{Np_str}.jpg'
    else:
        filename = f'SLICE/Budget_Updraft_Profiles_{res}_{t_res}_{Np_str}_SLICE.jpg'
    
    save_path = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Saved figure to: {save_path}")

In [ ]:
fig = plt.figure(figsize=(15, 7))
gs = fig.add_gridspec(1, 3)


vars = [
        'wb_hadv', 'wb_vadv', 'wb_hidiff', 'wb_vidiff',
        'wb_hturb', 'wb_vturb', 
        'wb_pgrad', 'wb_buoy'
    ]
budget_unit=r"$(m/s^2)$"
ax = fig.add_subplot(gs[0, 0])  # plot in bottom row full width
PlotBudgetProfiles(ax,vars,budget_unit)
plt.tight_layout()

vars = [
    'qvb_hadv', 'qvb_vadv', 'qvb_hidiff', 'qvb_vidiff',
    'qvb_hturb', 'qvb_vturb', 
    'qvb_mp'
]
budget_unit=r"$(g/kg/s)$"
ax = fig.add_subplot(gs[0, 1])  # plot in bottom row full width
PlotBudgetProfiles(ax,vars,budget_unit)
plt.tight_layout()

vars = [
    'ptb_hadv', 'ptb_vadv', 'ptb_hidiff', 'ptb_vidiff',
    'ptb_hturb', 'ptb_vturb', 
    'ptb_mp',# 'ptb_rad',
    #'ptb_div', 'ptb_diss'
]
budget_unit=r"$(K/s)$"
ax = fig.add_subplot(gs[0, 2])  # plot in bottom row full width
PlotBudgetProfiles(ax,vars,budget_unit)

fig.suptitle("Updraft BUDGET General (dashed) and Cloudy (solid) Updraft Domain Profiles ")
plt.tight_layout()

axes=fig.get_axes()
#FIXING TICKS
SnapLimitsToTicks(axes, dim='x')
for axis in axes:
    axis.set_ylim(0,20)
apply_scientific_notation(axes,decimals=2,scientific=True)

#ADDING AXVLINES
for ax in axes:
    ax.axvline(0,color='gray',linestyle='dashed',zorder=-10)

#FIX FIGURE SPACING
fig.subplots_adjust(top=0.92, wspace=0.2, hspace=0.4)

#SAVING
SavePlot(fig, res, t_res, Np_str, dir2)